In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
import glob
from PIL import Image

MODE = 'real'
MODEL_NAME = 'faster_rcnn_inception_v2_coco_' + MODE

In [2]:
PATH_TO_CKPT = '../ros/src/tl_detector/model/' + MODEL_NAME + '/frozen_inference_graph.pb'

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

class TLClassifier(object):
    def __init__(self):
        detection_graph = TLClassifier.load_graph()
        self.detection_graph = detection_graph
        self.sess = tf.Session(graph=detection_graph)
        self.image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        self.detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        self.detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = detection_graph.get_tensor_by_name('num_detections:0')


    @staticmethod
    def load_graph():
        detection_graph = tf.Graph()
        print("Loading graph...")
        with detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
        print("Graph loaded!")
        return detection_graph

    def get_classification(self, image):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)

        # Actual detection.
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})

        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes).astype(np.int32)

        if len(scores) < 1 or scores[0] < 0.4:
            return "unknown";

        cls = classes[0]
        print(cls, scores[0])
        
        box = boxes[0]
        box_h = box[2] - box[0]
        box_w = box[3] - box[1]
        
        if box_h < 0.03 or box_w < 0.01:  
            print("too small")
            return "unknown"

        print('bounding box: {} {}'.format(box_h, box_w))
        
        if cls == 1:
            return "red"
        elif cls == 2:
            return "yellow"
        elif cls == 3:
            return "green"

        return "unknown"

        
PATH_TO_TEST_IMAGES_DIR = 'test_images_' + MODE
CLASSES = ['red', 'yellow', 'green', 'unknown']

paths = []
for cls in CLASSES:
    paths += glob.glob(PATH_TO_TEST_IMAGES_DIR + "/" + cls + "/*.jpg")

tl = TLClassifier()

pass_count = 0
failure_count = 0

def cls_from_path(path):
    return path.split('/')[1]

for image_path in paths:
    image = Image.open(image_path)
    image_np = load_image_into_numpy_array(image)
    
    cls = cls_from_path(image_path)
    prediction_cls = tl.get_classification(image_np)
    if cls == prediction_cls:
        pass_count += 1
    else:
        failure_count += 1
    print("path={}, class={}\n".format(image_path, prediction_cls))


print("result: pass={}, failure={}".format(pass_count, failure_count))


Loading graph...
Graph loaded!
(1, 0.9990926)
bounding box: 0.0920414924622 0.0252014100552
path=test_images_real/red/carla1_frame0002.jpg, class=red

(1, 0.9997377)
bounding box: 0.194313883781 0.0502778589725
path=test_images_real/red/frame0114.jpg, class=red

(1, 0.99950874)
bounding box: 0.163211882114 0.0417515337467
path=test_images_real/red/frame0118.jpg, class=red

(1, 0.99943954)
bounding box: 0.140162020922 0.0388954281807
path=test_images_real/red/frame0126.jpg, class=red

(1, 0.8697241)
bounding box: 0.0761017352343 0.023460149765
path=test_images_real/red/frame0269.jpg, class=red

(1, 0.9990447)
bounding box: 0.0827355831861 0.0222415924072
path=test_images_real/red/frame0289.jpg, class=red

(1, 0.99902654)
bounding box: 0.0842112898827 0.0227828621864
path=test_images_real/red/frame0298.jpg, class=red

(1, 0.99640346)
bounding box: 0.0869619846344 0.0228321552277
path=test_images_real/red/frame0311.jpg, class=red

(1, 0.9998801)
bounding box: 0.171190917492 0.044294118881

In [3]:
accuracy = float(pass_count) / float(pass_count + failure_count)
print("accuracy={}".format(accuracy))

accuracy=1.0
